In [1]:
import sys
import numpy as np

In [2]:
import pandas as pd

In [3]:
from pathlib import Path

In [4]:
in_path = Path(r"C:\Data Analyst\Python\Project\sales_data.xlsx")
df = pd.read_excel(in_path)

In [5]:
df.columns

Index(['country', 'order_value_EUR', 'cost', 'date', 'category',
       'customer_name', 'sales_manager', 'sales_rep', 'device_type',
       'order_id'],
      dtype='object')

In [6]:
df.head()

,country,order_value_EUR,cost,date,category,customer_name,sales_manager,sales_rep,device_type,order_id
0,Sweden,98320.37,77722.25,8/23/2020,Games,Konopelski LLC,Maxie Marrow,Tarrah Castelletti,Tablet,70-0511466
1,France,46296.26,40319.41,5/15/2020,Games,Wisoky Inc,Othello Bowes,Amelina Piscopiello,Tablet,77-3489084
2,Portugal,140337.34,115708.14,2020-04-09 00:00:00,Appliances,Hegmann Group,Celine Tumasian,Corene Shirer,PC,65-8218141
3,France,203604.46,175344.16,6/26/2019,Electronics,Kirlin and Sons,Othello Bowes,Crysta Halls,Mobile,29-5478106
4,UK,63979.04,56032.84,10/22/2019,Games,Schoen-Keeling,Jessamine Apark,Genevra Charrisson,PC,27-3437546


In [7]:
df.columns = (df.columns.str.strip().str.lower()
              .str.replace(r"[^0-9a-zA-Z_]","_",regex = True)
              .str.replace(r"__+","_",regex = True))

In [8]:
#strip spaces, fill device_type
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()
df["device_type"] = df["device_type"].replace(["","None","nan"],
                                             np.nan).fillna("Unknown")

In [9]:
print(df.columns)

Index(['country', 'order_value_eur', 'cost', 'date', 'category',
       'customer_name', 'sales_manager', 'sales_rep', 'device_type',
       'order_id'],
      dtype='object')


In [10]:
#parse date + numerics
df["date"]=pd.to_datetime(df["date"],errors="coerce",
                               dayfirst = False)
df["order_value_eur"] = pd.to_numeric(df["order_value_eur"],
                                      errors="coerce")
df["cost"]=pd.to_numeric(df["cost"].astype(str).
                         str.replace(r"[^0-9\.-]","",
                                     regex=True),errors="coerce")

In [11]:
#metrics
df["profit_eur"] = df["order_value_eur"] - df["cost"]
df["profit_margin"] = np.where(df["order_value_eur"]>0,
                              df["profit_eur"]/df["order_value_eur"]
                               ,np.nan)

In [12]:
#reduplicate by order_id(keep last)
df = df.drop_duplicates(subset=["order_id"],keep='last')

In [13]:
#export
df.to_excel("sales_clean_full.xlsx",index=False)
fact = df.dropna(subset = ["order_value_eur","cost"])[
["order_id","date","country","category","customer_name",
 "sales_manager","sales_rep","device_type","order_value_eur",
 "cost","profit_eur","profit_margin"]
]
fact.to_csv("sales_fact_clean.csv",index=False)
print("Done")

Done


In [14]:
df.head()

,country,order_value_eur,cost,date,category,customer_name,sales_manager,sales_rep,device_type,order_id,profit_eur,profit_margin
0,Sweden,98320.37,77722.25,2020-08-23,Games,Konopelski LLC,Maxie Marrow,Tarrah Castelletti,Tablet,70-0511466,20598.12,0.2095
1,France,46296.26,40319.41,2020-05-15,Games,Wisoky Inc,Othello Bowes,Amelina Piscopiello,Tablet,77-3489084,5976.85,0.1291
2,Portugal,140337.34,115708.14,NaT,Appliances,Hegmann Group,Celine Tumasian,Corene Shirer,PC,65-8218141,24629.20,0.1755
3,France,203604.46,175344.16,2019-06-26,Electronics,Kirlin and Sons,Othello Bowes,Crysta Halls,Mobile,29-5478106,28260.30,0.1388
5,Italy,127075.41,110924.13,2019-09-24,Clothing,Labadie and Sons,Piggy Roscrigg,Joshua Prevost,PC,57-6602854,16151.28,0.1271


In [15]:
df.columns

Index(['country', 'order_value_eur', 'cost', 'date', 'category',
       'customer_name', 'sales_manager', 'sales_rep', 'device_type',
       'order_id', 'profit_eur', 'profit_margin'],
      dtype='object')

In [16]:
df.head()

,country,order_value_eur,cost,date,category,customer_name,sales_manager,sales_rep,device_type,order_id,profit_eur,profit_margin
0,Sweden,98320.37,77722.25,2020-08-23,Games,Konopelski LLC,Maxie Marrow,Tarrah Castelletti,Tablet,70-0511466,20598.12,0.2095
1,France,46296.26,40319.41,2020-05-15,Games,Wisoky Inc,Othello Bowes,Amelina Piscopiello,Tablet,77-3489084,5976.85,0.1291
2,Portugal,140337.34,115708.14,NaT,Appliances,Hegmann Group,Celine Tumasian,Corene Shirer,PC,65-8218141,24629.20,0.1755
3,France,203604.46,175344.16,2019-06-26,Electronics,Kirlin and Sons,Othello Bowes,Crysta Halls,Mobile,29-5478106,28260.30,0.1388
5,Italy,127075.41,110924.13,2019-09-24,Clothing,Labadie and Sons,Piggy Roscrigg,Joshua Prevost,PC,57-6602854,16151.28,0.1271


- 🔹 Step 1: Data Source (Excel → Python)
- File: sales data.xlsx
- Issue: messy (text in cost column, missing values, inconsistent dates).
- Python Cleaning Script does:
- Renames columns → snake_case (order_value_eur instead of order_value_EUR).
- Converts date → proper datetime.
- Converts order_value_eur & cost → numeric.
- Adds new columns:
- profit_eur = order_value_eur - cost
- profit_margin = profit_eur / order_value_eur
- Replaces missing device_type with "Unknown".
- Drops duplicate order_id.
- Output: sales_fact_clean.csv (clean dataset).

- 🔹 Step 2: Database (PostgreSQL)
- PostgreSQL = database where cleaned data lives.
- Why PostgreSQL?
- Industry standard for analytics.
- Handles large datasets better than Excel.
- Can be queried directly by BI tools like Power BI.
- DDL (Schema Creation)
n NUMERIC
);

- CREATE TABLE sales_orders (
- order_id VARCHAR PRIMARY KEY
- date DATE
- country VARCHAR
- category VARCHAR
- customer_name VARCHAR
- sales_manager VARCHAR
- sales_rep VARCHAR
- device_type VARCHAR
- order_value_eur NUMERIC
- cost NUMERIC
- profit_eur NUMERIC
- profit_margin NUMERIC
- );
- Loading Data
- Use COPY command or pgAdmin to import sales_fact_clean.csv into sales_orders.

- 🔹 Step 3: SQL Queries for Analysis
- 
-
- Total Revenue & Profit
- SELECT SUM(order_value_eur) AS total_revenue,
- SUM(profit_eur) AS total_profit
- FROM sales_orders;
- 
- Revenue by Country
- SELECT country, SUM(order_value_eur) AS revenue
- FROM sales_orders
- GROUP BY country
- ORDER BY revenue DESC;
- 
- 
- Monthly Revenue Trend
- SELECT DATE_TRUNC('month', date) AS month,
- SUM(order_value_eur) AS revenue
- FROM sales_orders
- GROUP BY month
- ORDER BY month;
- These queries validate insights before visualization.


- Step 4: Power BI Dashboard
- Connect Power BI → PostgreSQL (sales_orders).
- Build visuals:
- Cards → Total Revenue, Profit, Avg Margin.
- Line Chart → Monthly Revenue.
- Bar Chart → Revenue by Country.
- Pie Chart → Revenue Share by Category.
- Table → Detailed order info.
- Slicers → Date, Country, Device Type.

- 🔹 Step 5: Deliverables
- Python script (clean_sales_data.py) → reproducible ETL.
- PostgreSQL schema & queries (sales_orders.sql).
- Power BI dashboard (sales_dashboard.pbix).
- Documentation (README.md) → explains workflow & insights

- Story-Like Explanation (Interview Style)
- 👉 Imagine you’re explaining this to an interviewer:
"Our company had sales data sitting in Excel, but it was messy and hard to analyze. My goal was to build an end-to-end pipeline that transforms this - data into business insights.

First, I wrote a Python script to clean the Excel file. For example, the cost column was stored as text, dates weren’t properly formatted, and some device types were missing. I standardized everything, created new fields like Profit and Profit Margin, and exported a clean dataset.

Next, I loaded this data into PostgreSQL. I designed a proper schema (sales_orders) with numeric and date types. Now our data was in a central, structured database instead of a flat Excel file.

Using SQL queries, I validated key insights: total revenue, monthly revenue trends, revenue by country, and profit margins by product category. This gave me confidence the data was ready for visualization.

Finally, I connected Power BI directly to PostgreSQL and built an interactive dashboard. It has high-level KPIs (Revenue, Profit, Margin), trend charts (Monthly Revenue), comparison charts (Revenue by Country, Category Share), and filters (Date, Device Type, Country). Managers can now slice the data however they want, in real time.

This project demonstrates a complete workflow: ETL with Python → Data Storage in PostgreSQL → Analysis with SQL → Visualization with Power BI. It’s a real-world setup used in modern data teams, and it converts raw messy data into clear business decisions."